In [1]:
'''
Import necessary library
'''

import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [2]:
'''
Read input data
'''
train_data = pd.read_csv('training_data.csv',header=None)
test_data = pd.read_csv('testing_data.csv',header=None)
train_size = len(train_data)
test_size = len(test_data)
print(train_size,test_size)

(47438, 25545)


In [3]:
'''
Deal with input data
'''
test_index = test_data.loc[:,0]
test_data = test_data.drop(0,axis = 1)
def test_rename(x):
    return x+1
test_data = test_data.rename(mapper=test_rename,axis=1)
train_label = train_data.iloc[:,1]
train_data_d = train_data.drop(labels=[0,1],axis=1)

In [4]:
concated_df = pd.concat((train_data_d,test_data))

In [5]:
cat_feature = [2,5,6,7,8,9,10,11,13,14,15,24,25,27]
num_feature = [3,4,12,16,17,18,19,20,21,22,23,26,28]

In [6]:
'''
Deal with missing value
Fill category feature with mode, numeric feature with mean
'''
for col in cat_feature:
    concated_df[col] = concated_df[col].fillna(concated_df[col].mode()[0])
for col in num_feature:
    concated_df[col] = concated_df[col].fillna(concated_df[col].mean())
concated_df = pd.get_dummies(concated_df,columns=cat_feature)

In [7]:
train_data_d = concated_df.iloc[:train_size,:]
test_data = concated_df.iloc[train_size: , : ]
print(len(train_data_d),len(test_data) )

(47438, 25545)


In [8]:
'''
Train/test dataframe into numpy array
'''
X = train_data_d.values
y = train_label.values
X_test = test_data.values

In [9]:
'''
Standardlize numeric feature
'''
std = StandardScaler()
std.fit(X[:,:14])
X[:,:14] = std.transform( X[:,:14] )
X_test[:,:14] = std.transform(X_test[:,:14])

In [10]:
from sklearn.decomposition import PCA 
pca=PCA(n_components=23)
pca.fit(X)
newData=pca.transform(X)
newtestData=pca.transform(X_test)

In [11]:
rus = RandomUnderSampler(random_state = 0)
newData, y = rus.fit_sample(newData, y)

In [12]:
from sklearn.model_selection import cross_val_score

In [13]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=23)
knn.fit(newData, y)
cross_val_score(knn, newData, y, cv=5)
'''
Make prediction
'''
pred = knn.predict(newtestData)

In [14]:
'''
Output prediction
'''
with open('super_easy_baseline.csv','w') as f:
    f.write('Id,Prediction\n')
    for idx , p in zip(test_index , pred):
        f.write('{},{}\n'.format(idx,p))